In [ ]:
# !unzip EuroSAT_RGB_dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2768.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2769.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_277.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2791.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2792.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_280.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_281.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2815.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2820.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2821.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2825.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2826.jpg  
  inflating: EuroSAT_RGB_dataset/val/AnnualCrop/AnnualCrop_2827.jpg  
  inflating: EuroSAT_RGB_dataset/val/Annua

In [14]:
import os
import argparse
from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [15]:
data_dir = "EuroSAT_RGB"  # Use the original dataset with all classes in one place
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

# Perform transformation on the EuroSAT images
transform = transforms.Compose([
    transforms.Resize((224, 224)),                                      # resizes the image to 224x224
    transforms.RandomHorizontalFlip(),                                  # performs horizontal flip of the input
    transforms.ToTensor(),                                              # Convets to Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # normalizes the channels [R, G, B], [mean, std_dev]
])

Using device:  cpu


In [16]:
# Dataset -----------------------------------------------------------------------------------------------------------------------------------
dataset = datasets.ImageFolder(root= data_dir, transform = transform)
# num_classes = len(dataset) # Incorrectly set to the number of images
num_classes = len(dataset.classes) # Correctly set to the number of classes
print("No. of classes: ", num_classes)

val_size = int(0.2 * len(dataset))
train, val = random_split(dataset, [len(dataset)-val_size, val_size])   # [train_size, val_size]

train_loader = DataLoader(train, batch_size=64, shuffle=True)
val_loader = DataLoader(val, batch_size=64, shuffle=True)

No. of classes:  10


In [13]:
# Model --------------------------------------------------------------------------------------------------------------------------------------
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)  # Using pretrained ResNet50 as recommended by the paper
model.fc = nn.Linear(model.fc.in_features, num_classes)                 # changing the last layer
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:05<00:00, 19.4MB/s]


In [12]:
# Training -------------------------------------------------------------------------------------------------------------------------------------
total_epochs = 100
train_size = len(train) # Corrected train_size calculation

# Learning rate scheduler (as mentioned in the paper)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

for epoch in range(total_epochs):
    model.train()
    train_loss = 0
    train_correct = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)                                       # forward pass
        loss = criterion(outputs, labels)                           # compute loss
        loss.backward()                                             # backward pass
        optimizer.step()                                            # update parameters

        train_loss += loss.item() * imgs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    train_acc = train_correct / train_size                          # computes accuracy

    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1} Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * imgs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    val_acc = val_correct / val_size

    print(f"Epoch {epoch+1}/{total_epochs} "
          f"Train Acc: {train_acc:.4f} Val Acc: {val_acc:.4f}")

    # Step the scheduler
    scheduler.step()


torch.save(model.state_dict(), "resnet50_eurosat_rgb.pth")
print("Model saved to resnet50_eurosat_rgb.pth!!")

NameError: name 'optimizer' is not defined

In [ ]:
# Install PyTorch with CUDA support
import subprocess
import sys

print("Installing PyTorch with CUDA 12.1 support...")
try:
    # Uninstall existing CPU-only versions
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "torch", "torchvision", "torchaudio", "-y"], 
                   check=True, capture_output=True, text=True)
    print("Uninstalled existing PyTorch versions")
    
    # Install CUDA-enabled versions
    result = subprocess.run([
        sys.executable, "-m", "pip", "install", 
        "torch", "torchvision", "torchaudio", 
        "--index-url", "https://download.pytorch.org/whl/cu121"
    ], check=True, capture_output=True, text=True)
    print("Successfully installed PyTorch with CUDA support!")
    print("Please restart the kernel to use the new PyTorch version.")
    
except subprocess.CalledProcessError as e:
    print(f"Installation failed: {e}")
    print("Error output:", e.stderr)

Installing PyTorch with CUDA 12.1 support...
Uninstalled existing PyTorch versions
Uninstalled existing PyTorch versions
